In [1]:
import He2021_v2_4 as H
import numpy as np
from scipy.spatial import Delaunay
import matplotlib.pyplot as plt
import matplotlib
import cv2
from astropy.io import fits
import pandas as pd
import random as r
import lenstronomy.Util.constants as c
import png
import pylab
import compose_class
from astropy.cosmology import Planck18 as cosmo
from lenstronomy.Cosmo.lens_cosmo import LensCosmo

import matplotlib.pyplot as plt
from lenstronomy.LensModel.lens_model import LensModel
import lenstronomy.Util.param_util as param_util
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver

## 构建透镜系统

In [4]:
#导入前景和背景所需要的红移（这一版本中，前景仅使用100个有图像的cmass）
cmass_df = pd.read_csv('/nfsdata/users/zizhao-tulip/cata_Dec/decals_dev_cmass_dropnan_colrename_first100.csv')
eBOSS_df = pd.read_csv('/nfsdata/users/zizhao-tulip/cata_Dec/eBOSSDR12_QSO_catlogue_OnlyQSO.csv')

cmass_df = cmass_df[cmass_df["redshift"]>0]
eBOSS_df = eBOSS_df[eBOSS_df["Z_PIPE"]>0]

num_of_eBOSS = len(eBOSS_df)
num_of_cmass = len(cmass_df)

eBOSS_df.index = range(0,num_of_eBOSS)
cmass_df.index = range(0,num_of_cmass)

flag_abondon = 0

#设定lens/source平面的边界
scale = 10
xmin = -scale
xmax = scale
ymin = -scale
ymax = scale

#设定格子的大小
pixel_size = 0.1

#所模拟巡天的视差，这个用来切stamp
FoV = 20              #这个用来降低分辨率，pixel_size是计算用到的分辨率，pixel_decals是保存图片的分辨率
pixel_decals = 0.1

#两个像之间的放大率倍数不能超过这个
magratio_threshold = 10

#前景在pixel_decals下的大小
foreground_width_shoudbe = round(FoV/pixel_decals/2)

#PSF水平
g_psf_fwhm_decals = 1.26662
r_psf_fwhm_decals = 1.13551
z_psf_fwhm_decals = 1.11053 

#给出透镜中心位置
center_x = 0.0
center_y = 0.0


#只要像的个数为2、4的mock
possible_img_num = [2,4]  

def WrtLne(Line,f,flag):
    f.writelines(Line)
    if flag:
        f.write('\n')

/tmp/ipykernel_1653599/489006039.py:3: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  eBOSS_df = pd.read_csv('/nfsdata/users/zizhao-tulip/cata_Dec/eBOSSDR12_QSO_catlogue_OnlyQSO.csv')


In [5]:
for ii in range(0,10):
    
    flag_abondon = 0
    
    #选择前景\背景
    cmass_index = r.randint(0,num_of_cmass-1)
    eBOSS_index = r.randint(0,num_of_eBOSS-1)

    while eBOSS_df["Z_PIPE"][eBOSS_index]<cmass_df["redshift"][cmass_index]:  
        eBOSS_index = r.randint(0,num_of_eBOSS-1)


    sigma_v = H.Appmag2Sigmav(cmass_df["redshift"][cmass_index],
                           eBOSS_df["Z_PIPE"][eBOSS_index],
                           cmass_df["rg_DEV"][cmass_index],
                           cmass_df["fluxr"][cmass_index])

    ######################################################################
    lens_cosmo = LensCosmo(z_lens=cmass_df["redshift"][cmass_index], z_source=eBOSS_df["Z_PIPE"][eBOSS_index],                            cosmo=cosmo)
    theta_E = lens_cosmo.sis_sigma_v2theta_E(sigma_v/1000)[0][0]
    ######################################################################

    #依据爱因斯坦半径计算结果，判断是否要重新选择背景
    num_of_while = 0
 
    while theta_E < 0.5:
        eBOSS_index = r.randint(0,num_of_eBOSS-1)
        lens_cosmo = LensCosmo(z_lens=cmass_df["redshift"][cmass_index], z_source=eBOSS_df["Z_PIPE"][eBOSS_index],                            cosmo=cosmo)
        theta_E = lens_cosmo.sis_sigma_v2theta_E(sigma_v/1000)[0][0]
        print("due to small theta_E, the QSO is reselected.")
        num_of_while += 1
        if num_of_while > 50:
            print("this zLens is too high(", round(cmass_df["redshift"][cmass_index],2), ") to find a suitable QSO.")
            flag_abondon = 1
            break
    
    #如果找不到就放弃这个循环，开始下一个
    if flag_abondon:
        continue

    #计算轴比
    e1 = cmass_df['shapedev_e1'][cmass_index]
    e2 = cmass_df['shapedev_e2'][cmass_index]
    e = np.sqrt(e1**2 + e2**2)
    qlens = (1-e)/(1+e)

    #给定方位角
    phi_deg = np.random.rand()*180

    #综合上述几步计算结果，给出完备透镜参数
    lpar = np.array( [
        theta_E,   #eninstein radius
        center_x,  #xcenter
        center_y,  #ycenter
        phi_deg,   #position angle, +Y轴为0度
        qlens,     #axis ratio
    ])

    #把lens\source平面按照pixel size大小做分割
    plane_xx,plane_yy = np.mgrid[xmin:xmax+pixel_size:pixel_size, ymin:ymax+pixel_size:pixel_size]
    plane_xx[plane_xx==0] = 1e-8
    plane_yy[plane_yy==0] = 1e-8

    #得到caustic和critical
    miu_inversed = H.GetInversedMiu(plane_xx,plane_yy,pixel_size, lpar)
    critical_inner, critical_outer, caustic_inner, caustic_outer = H.GetCritialCastic(plane_xx,plane_yy, miu_inversed,lpar)

    while True:
        
        beta_ra, beta_dec = H.GetSourcePosition(caustic_outer)
        beta_ra = beta_ra
        beta_dec = beta_dec
        beta_ra_dec = np.array([beta_ra, beta_dec])

        
        plane_xx, plane_yy = np.mgrid[xmin:xmax+pixel_size:pixel_size, ymin:ymax+pixel_size:pixel_size]
        img_plane_grid = np.c_[plane_xx.ravel(),plane_yy.ravel()]
        tri_img_plane_Delaunay = Delaunay(img_plane_grid)
        tri_img_plane = img_plane_grid[tri_img_plane_Delaunay.simplices]
        tri_source_plane = H.TriangleImg2Source(tri_img_plane, lpar)
        index_satisfied = H.GetIndexOfSatisfiedTriangle(tri_source_plane, beta_ra_dec)
        index_satisfied = H.DropCntrImg(tri_img_plane, index_satisfied, pixel_size)
        
        cond1 = len(index_satisfied) not in possible_img_num
        if cond1:
            print('num of imgs is ', len(index_satisfied),' resample the beta x/y.')
            continue
            
        tri_source_plane_satisfied = tri_source_plane[index_satisfied]
        tri_img_plane_satisfied = tri_img_plane[index_satisfied]
        img_ra_dac_list = []
        for idx in range(len(tri_img_plane_satisfied)):
            (imgx_x, imgx_y), iter_times = H.GetImgPositionOfOneImg(tri_img_plane, index_satisfied[idx], lpar, beta_ra_dec)
            img_ra_dac_list.append([imgx_x, imgx_y])
        
        
        cond2 = ['Catch a bug', 'Catch a bug'] in img_ra_dac_list
        if cond2:
            print('position program report a bug, resample the beta x/y.')
            continue
        miu_list = []

        for img_x, img_y in img_ra_dac_list:
            miu_list.append(abs(H.GetMagAnlytcl(img_x, img_y, lpar)))
            
        cond3 = max(miu_list)/min(miu_list) > magratio_threshold
        if cond3:
            print('reach miuratio threshold, resample the beta x/y.')
            continue
        if (not cond1) & (not cond2) & (not cond3):
            break

    miu_arr = np.array(miu_list)
    miu_sort_arr = np.sort(np.array(miu_list))
    img_ra_dac_sort_arr = np.array([])
    for idx in range(0,len(miu_arr)):
        this_crdnt = np.array(img_ra_dac_list[np.where(miu_sort_arr[idx]==miu_arr)[0][0]])
        img_ra_dac_sort_arr = np.append(img_ra_dac_sort_arr, this_crdnt)
        
    #汇总已有信息，输出 & 写入文件
    lens_para_log = str(ii+1)+"\n"
    lens_para_log = lens_para_log+"q:              "+str(lpar[4])+"\n"+"zd:             "+str(cmass_df["redshift"][cmass_index])+"\n"+                    "zs:             "+str(eBOSS_df["Z_PIPE"][eBOSS_index])+"\n"
    lens_para_log = lens_para_log+"rng:            "+str(lpar[0])+"\n"+"phi:            "+str(lpar[3])+"\n"
    lens_para_log = lens_para_log+"sgm:            "+str(sigma_v[0][0]/1000)+"\n"
    img_log = "img_crdnt:      "+str(img_ra_dac_sort_arr)+"\n"
    miu_log = "miu:            "+str(miu_sort_arr)+"\n"
    qso_log = "QSO_crdnt:      "+str(beta_ra_dec)+"\n"
    msg_bfr_err = lens_para_log + miu_log + qso_log + img_log
    
    print(msg_bfr_err)
    f=open("5000tst_230614.txt","a")
    WrtLne([],f,True)
    WrtLne(msg_bfr_err,f,False)
    f.close()

    #lenstronomy
    lens_model_list = ['SIE']
    lensModel = LensModel(lens_model_list=lens_model_list, z_lens=cmass_df["redshift"][cmass_index], z_source=eBOSS_df["Z_PIPE"][eBOSS_index])
    e1_new, e2_new = param_util.phi_q2_ellipticity(phi=(phi_deg+90)/180.0*np.pi, q=qlens)
    kwargs_sie = {'theta_E':theta_E , 'e1': e1_new, 'e2': e2_new, 'center_x': 0.0, 'center_y': 0.0}
    kwargs_lens = [kwargs_sie]

    solver = LensEquationSolver(lensModel)
    theta_ra, theta_dec = solver.image_position_from_source(beta_ra, beta_dec, kwargs_lens)

    miu_lnstrnmy_arr = np.array([])
    for ra,dec in zip(theta_ra, theta_dec):
        miu_lnstrnmy_arr = np.append(miu_lnstrnmy_arr, abs(lensModel.magnification(ra,dec, kwargs_lens)))
    miu_lnstrnmy_sort_arr = np.sort(miu_lnstrnmy_arr)
    img_ra_dac_lnstrnmy_sort_arr = []
    for idx in range(0,len(miu_lnstrnmy_sort_arr)):
        this_ra  = np.array(theta_ra[np.where(miu_lnstrnmy_sort_arr[idx]==miu_lnstrnmy_arr)[0][0]])
        this_dec = np.array(theta_dec[np.where(miu_lnstrnmy_sort_arr[idx]==miu_lnstrnmy_arr)[0][0]])
        img_ra_dac_lnstrnmy_sort_arr = np.append(img_ra_dac_lnstrnmy_sort_arr, [this_ra,this_dec])

    if len(miu_lnstrnmy_arr) == len(miu_arr):
        #进行误差计算
        mag_err = abs(miu_lnstrnmy_sort_arr - miu_sort_arr).sum()/miu_lnstrnmy_sort_arr.sum()
        pos_err = np.sqrt(((img_ra_dac_lnstrnmy_sort_arr-img_ra_dac_sort_arr)**2).sum())/pixel_size
        tst_rslt = "miu_lnsr:       "+str(miu_lnstrnmy_sort_arr)+               "\nmag_err:        "+str(mag_err)+"\npstn_err(pxl):  "+str(pos_err)
    
        if mag_err>1 or pos_err>0.01:
            tst_rslt=tst_rslt+";error BEYOND thrshld"
        else:
            tst_rslt=tst_rslt+";NORMAL"
    else:
        tst_rslt = "Differed in nimg. \n"+\
                   "miu_lnsr:       "+str(miu_lnstrnmy_sort_arr)+"\n"+\
                   "img_crdnt_lnsr: "+str(img_ra_dac_lnstrnmy_sort_arr)
                    
    msg_ftr_err = tst_rslt
    
    
    #汇总误差信息，输出 & 写入文件
    print(msg_ftr_err)
    f=open("5000tst_230614.txt","a")
    WrtLne(msg_ftr_err,f,True)
    f.close()


due to small theta_E, the QSO is reselected.
due to small theta_E, the QSO is reselected.
1
q:              0.7645289961949497
zd:             0.51775
zs:             3.099798
rng:            0.797737450128555
phi:            61.93280567991554
sgm:            199.34349139753726
miu:            [0.96226114 5.4891806 ]
QSO_crdnt:      [ 0.29953193 -0.09606602]
img_crdnt:      [-0.41515274  0.16052831  1.06259181 -0.17578659]

miu_lnsr:       [0.96226098 5.48918577]
mag_err:        8.269121274307756e-07
pstn_err(pxl):  5.39898284650213e-06;NORMAL
2
q:              0.8263887478472022
zd:             0.63643
zs:             2.4596400000000003
rng:            2.058445326487938
phi:            169.74110725994686
sgm:            345.8185836627031
miu:            [0.41756825 3.01121537]
QSO_crdnt:      [0.5312768  1.22177759]
img_crdnt:      [-0.24003398 -0.61632843  1.4957016   2.97396596]

miu_lnsr:       [0.41756895 3.01122862]
mag_err:        4.069467745566398e-06
pstn_err(pxl):  6.18231442

/home/zizhao/palace/He2021_v2_4.py:245: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  nearby_tri_indexs = np.array([fisrt_tri_index, second_tri_index, thrid_tri_index, np.array([index_focus])])


4
q:              0.48943116963889843
zd:             0.60065
zs:             2.4229395
rng:            0.5246373137765805
phi:            38.680835451388376
sgm:            172.2924887440106
miu:            [0.36692959 2.18130143]
QSO_crdnt:      [-0.03507406  0.28532181]
img_crdnt:      [ 0.06901474 -0.16657995  0.19940033  0.76928813]

miu_lnsr:       [0.36692962 2.18130119]
mag_err:        1.0516682766176536e-07
pstn_err(pxl):  6.565390546201927e-06;NORMAL
5
q:              0.7426087611972509
zd:             0.5717800000000001
zs:             2.5617635
rng:            0.7792586966899016
phi:            92.8413230838479
sgm:            205.69391399614116
miu:            [1.20996917 7.90289233]
QSO_crdnt:      [-0.24209708  0.02256371]
img_crdnt:      [ 0.49337311 -0.01717631 -0.93259176  0.31247762]

miu_lnsr:       [1.20997044 7.90287887]
mag_err:        1.6171865446876945e-06
pstn_err(pxl):  4.750533292081282e-06;NORMAL
6
q:              0.7750071612781277
zd:             0.53682


## 计算timedelay

In [6]:
#mine prgram
H.TwoImgTimeDelay(img_ra_dac_sort_arr, lpar,\
          zd=cmass_df["redshift"][cmass_index], zs=eBOSS_df["Z_PIPE"][eBOSS_index])

(94.32664775990072, 0.0)

In [7]:
#lenstronomy
fermat_lstrnmy_A = lensModel.fermat_potential(img_ra_dac_lnstrnmy_sort_arr[0], img_ra_dac_lnstrnmy_sort_arr[1], kwargs_lens)
fermat_lstrnmy_B = lensModel.fermat_potential(img_ra_dac_lnstrnmy_sort_arr[2], img_ra_dac_lnstrnmy_sort_arr[3], kwargs_lens)
print(lens_cosmo.time_delay_units(fermat_lstrnmy_A)-\
      lens_cosmo.time_delay_units(fermat_lstrnmy_B))

94.32662667608443
